In [1]:
!pip install langchain accelerate peft

**Load Fine-tuned Model**

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BartForConditionalGeneration
from peft import PeftModel
import torch

base_model = "fnlp/bart-base-chinese"
new_model = "tonyma163/bart_v1"

device="cuda:0"

base_model_reload = BartForConditionalGeneration.from_pretrained(
        base_model,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map=device,
        #trust_remote_code=True,
)
base_model_reload.half()

model = PeftModel.from_pretrained(base_model_reload, new_model)

2024-04-17 15:52:49.817720: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-17 15:52:49.817777: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-17 15:52:49.819394: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(base_model, trust_remote_code=True)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [4]:
from transformers import Text2TextGenerationPipeline

pipe = Text2TextGenerationPipeline(model=model, tokenizer=tokenizer)

The model 'PeftModelForSeq2SeqLM' is not supported for . Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


In [ ]:
pipe('周迅的星座是什么？')

**Loading Document**

In [5]:
import pandas as pd
import ast

file_path = "/kaggle/input/nlp-knowledge-set/knowledge_set.txt"

data = []

# Open the file and parse each line from string to tuple
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        if line.strip():  # Ensure the line is not empty
            try:
                # Convert string representation of tuple to actual tuple
                tuple_data = ast.literal_eval(line.strip())
                data.append(tuple_data)
            except SyntaxError:
                print(f"Skipping malformed line: {line.strip()}")

# Load the data into a DataFrame
df = pd.DataFrame(data, columns=['Entity', 'Category', 'Answer'])

In [ ]:
df.head()

**Knowledge Graph**

In [6]:
import networkx as nx
import pandas as pd

# Create a directed graph
G = nx.DiGraph()

# Add nodes and edges based on the DataFrame
for index, row in df.iterrows():
    entity_node = f"Entity: {row['Entity']}"
    category_node = f"{entity_node}, Category: {row['Category']}"
    answer_node = f"Answer: {row['Answer']}"

    # Add nodes and edges
    G.add_node(entity_node, type='Entity')
    G.add_node(category_node, type='Category', entity=row['Entity'])
    G.add_node(answer_node, type='Answer', category=row['Category'])
    
    G.add_edge(entity_node, category_node)
    G.add_edge(category_node, answer_node)

In [7]:
# Example query: Find all answers linked to a specific entity and category
entity_query = '西宁'
category_query = '2018-11-14'

processed_entity_query = f"Entity: {entity_query}"
processed_category_query = f"{processed_entity_query}, Category: {category_query}"

# First, find the category node directly connected to the entity
if (processed_entity_query, processed_category_query) in G.edges:
    answers = [node for node in G.successors(processed_category_query) if G.nodes[node]['type'] == 'Answer']
    for answer in answers:
        print(answer)
else:
    print("No such category for the given entity or wrong category/entity combination.")

Answer: 阴,东风,最高气温:5℃,最低气温:-4℃


**Find the Entity Only**

In [8]:
# Entity Only
def query_by_entity(graph, entity):
    # Create a comprehensive list to store results
    results = []

    # Construct the entity node identifier
    entity_node = f"Entity: {entity}"

    # Check if the entity node exists in the graph
    if entity_node in graph:
        # Get all category nodes linked to the entity
        category_nodes = [node for node in G.successors(entity_node) if G.nodes[node]['type'] == 'Category']
        
        for category in category_nodes:
            # Retrieve all answer nodes linked to this category
            answer_nodes = [node for node in G.successors(category) if G.nodes[node]['type'] == 'Answer']
            results.append((category, answer_nodes))
    else:
        print(f"No information available for entity: {entity}")
    
    return results

In [9]:
# Query the graph for a specific entity
entity = "周迅"
results = query_by_entity(G, entity)

# Print the results
if results:
    print(f"Information for entity '{entity}':")
    for category, answers in results:
        print(f"\n{category}:")
        for answer in answers:
            print(f"  - {answer}")
else:
    print("No results found for the queried entity.")


Information for entity '周迅':

Entity: 周迅, Category: 评论:
  - Answer: 灵气逼人，倔犟，聪慧
  - Answer: 我的妈呦这土boqi嗓子
  - Answer: 每一个形象都演活了。
  - Answer: 竟然和我喜欢的刘若英是好友，果然都是小资
  - Answer: 在天朝唯一喜欢的演员
  - Answer: 喜欢她的身体散发出的各种灵气，甚至是她的嗓音·
  - Answer: 中国大陆目前演技最好的演员。
  - Answer: 到现在都还有灵气，真的很难得
  - Answer: amazinglady
  - Answer: 演技、容貌、修养，三方面俱佳，缺点太少。
  - Answer: 看完听风者终于坚定了支持周迅的决心~~
  - Answer: 演技和纯真的完美结合，不做作，不矫情
  - Answer: 小精灵，很享受她的美
  - Answer: 太喜欢了一直很喜欢。
  - Answer: #印象周迅#孙纳和张学宁
  - Answer: 美人~~~对演艺事业有强烈追求的演员~~
  - Answer: 挺喜欢她的~觉得演什么都像
  - Answer: 借用一句台词，“怎么会有如此好德又好色的人呢？”
  - Answer: 目前内地唯一有精湛演技的女生！
  - Answer: 这样的女人真的很别致
  - Answer: 公子！我宣你！我也宣萱！
  - Answer: #印象周迅#周公子生日快乐
  - Answer: 内地最好的女演员，没有之一。我说的是演技
  - Answer: 早期其实并不喜欢你……看了《李米的猜想》才喜欢上了你
  - Answer: 就是喜欢你。演技超好，脸蛋真漂漂。
  - Answer: 为什么打九分看看，《李米的猜想》
  - Answer: 大爱周迅！美丽、灵！
  - Answer: 有灵气的演员，喜欢她的笑
  - Answer: 最喜欢有独特气质的人了
  - Answer: 周迅，中国真正好的女演员
  - Answer: 我从来不知道什么叫淑女，更不装，我活的随意！
  - Answer: 她身上有股子灵气，学是学不来的~~~
  - Answer: 从大明宫词开始就很喜欢你那聪明空灵的劲。。。
  - Answe

**Document Splitting, Embedding, and Vector Store**

**Query Engine**

**Query the LLM**

In [10]:
def retrieve_from_graph(graph, query_entity):
    # This function will be similar to the earlier function but tailored for RAG integration
    entity_node = f"Entity: {query_entity}"
    relevant_data = []

    if entity_node in graph:
        category_nodes = [node for node in G.successors(entity_node) if G.nodes[node]['type'] == 'Category']
        for category in category_nodes:
            answers = [node for node in G.successors(category) if G.nodes[node]['type'] == 'Answer']
            relevant_data.extend(answers)
    return relevant_data

In [11]:
def generate_response(input_query, graph):
    # Retrieve relevant information from the graph
    retrieved_data = retrieve_from_graph(graph, input_query)
    context = " ".join(retrieved_data)
    print(f"context: {context}\n")
    # Combine the input query with the retrieved context
    combined_input = f"Query: {input_query} Context: {context}"
    
    #return tokenizer.decode(output_ids[0], skip_special_tokens=True)
    inputs = tokenizer.encode(combined_input, return_tensors="pt")
    outputs = model.generate(input_ids=inputs.to(device), max_new_tokens=126)
    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return {"response": response_text}

In [17]:
input_query = "郭富城"
response = generate_response(input_query, G)
print(response)

context: Answer: 人气天王 Answer: 娱圈高调炫富土豪明星 Answer: 名人 Answer: 那些爱穿裙子的男星 Answer: 男神级明星 Answer: 香港重量级明星 Answer: 广告商的宠儿 Answer: 那些腿短的男神们 Answer: 歌神 Answer: 大眼明星 Answer: 金马奖影帝 Answer: 明星成名前的打工经历 Answer: 爱车之人 Answer: 娱乐圈的钻石王老五 Answer: 天蝎座男明星 Answer: 香港歌坛天王 Answer: “天王” Answer: 心细 Answer: 浪漫 Answer: 香港男星 Answer: 性感 Answer: 中国知名影星 Answer: 古装剧中帅气男演员 Answer: 香港歌手 Answer: 超级巨星 Answer: “侦探” Answer: 亚洲天王 Answer: 大器晚成的影帝 Answer: 娱乐圈内的钻石王老五 Answer: 那些被结婚的明星们 Answer: 明星的初恋情人 Answer: 香港影帝 Answer: 国际巨星 Answer: 舞神 Answer: 老牌明星 Answer: 演艺圈的劳模 Answer: 观众眼中的梦中情人 Answer: 很低调的人 Answer: 联合国儿童基金会大使 Answer: 大男孩 Answer: 娱乐圈有钱任性土豪明星 Answer: 天王巨星 Answer: 很有才华的艺人 Answer: 演员 Answer: 细心 Answer: 酷劲十足 Answer: 那些颜值超高的赛车男神 Answer: 创造吉尼斯纪录的明星们 Answer: 港星 Answer: “舞台王者” Answer: 一众艺人 Answer: 那些高调分手的娱乐圈明星 Answer: 影帝 Answer: 国际影星 Answer: 网友心中的土豪明星 Answer: 娱乐圈的肌肉型男 Answer: 先锋人物 Answer: 肌肉型男 Answer: 舞台型歌手 Answer: “表演艺术家” Answer: 香港人气歌手 Answer: 影星 Answer: 大好人 Answer: 那些恐婚症男星 Answer: 运动品牌的明星代言人 Answer: 娱乐圈的恐婚男星 Answer: 纯真 Answer: "

/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [46,0,0], thread: [32,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [46,0,0], thread: [33,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [46,0,0], thread: [34,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [46,0,0], thread: [35,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [46,0,0], thread: [36,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [46,0,0], thread: [37,0,0] Assertion `srcIn

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
